In [ ]:
!pip install blueno

In [ ]:
from blueno import blueprint, Blueprint, DataFrameType,job_registry, create_pipeline


In [ ]:
import random
import time

import polars as pl


RAND_SIZE = 10
base_path = "/lakehouse/default/Tables"

In [ ]:
@blueprint(table_uri=f"{base_path}/bronze/products")
def bronze_products() -> DataFrameType:
    df = pl.DataFrame(
        {
            "product_id": [1, 2, 3],
            "product_name": ["ball", "bat", "tent"],
            "price": [4.99, 9.99, 29.99],
        }
    )

    time.sleep(random.random() * 2)

    return df

In [ ]:
@blueprint(
    table_uri=f"{base_path}/silver/products",
    primary_keys=["product_id"],
)
def silver_products(self: Blueprint, bronze_products: DataFrameType) -> DataFrameType:
    df = bronze_products.unique(subset=self.primary_keys)
    time.sleep(random.random() * RAND_SIZE)

    return df

In [ ]:
@blueprint(
    table_uri=f"{base_path}/gold/products",
    primary_keys=["product_id"],
)
def gold_products(self: Blueprint, silver_products: DataFrameType) -> DataFrameType:
    df = silver_products.filter(pl.col("product_id").is_not_null())

    return df

In [ ]:
tasks = create_pipeline(job_registry.jobs.values())

In [ ]:
job_registry.run(activities=tasks, concurrency=1)